In [1]:
import abc
import ollama

class Agent(abc.ABC):
    def __init__(self):
        self.sys_message = {'role': 'system', 'content': self.system_prompt()}
        self.user_message_default = "Introduce yourself."

    @abc.abstractmethod
    def system_prompt(self):
        """Return the system prompt content"""
        pass
    
    @abc.abstractmethod
    def user_prompt(self):
        pass

    def chat(self, user_input=None, model="gemma3n"):

        if user_input:
            user_content = self.user_prompt(user_input)
        else:
            user_content = self.user_message_default

        user_message = {'role': 'user', 'content': user_content}

        response = ollama.chat(model=model, messages=[self.sys_message, user_message])
        return response['message']['content'].strip()


In [2]:
class Preprocessor(Agent):
    def system_prompt(self):
        return "You are an expert editor of Tibetan Buddhist texts. You preprocess Tibetan material for translation."
    
    def user_prompt(self, user_input):
        return f"""Preprocess the following: {user_input}
                """

In [3]:
class Translator(Agent):
    def system_prompt(self):
        return "You are an expert translator of Tibetan Buddhist texts. You translate clearly and concisely, preserving the original intent of the text."
    
    def user_prompt(self, user_input):
        return f"""Translate the following from Tibetan to English:{user_input['source']}
                Consider these notes from the preprocessor: {user_input['prepro_notes']}"""


In [4]:
class Evaluator(Agent):
    def system_prompt(self):
        return "You are an expert in translation evaluation. You score translations based on fluency, literary quality, and meaning preservation."
    
    def user_prompt(self, user_input):
        return f"""Score the following translation on a continuous scale from 0 to 100, where score of zero means
                "no meaning preserved" and score of one hundred means "perfect meaning and grammar".

                Source: {user_input['source']}
                Target: {user_input['target']}
                """

In [5]:
class PostEditor(Agent):
    def system_prompt(self):
        return """"You are an expert translation editor. You provide improved translations based on an original translation and a score from an evaluator. 
        You preserve the good qualities of the first draft but improve clarity and fluency."""
    
    def user_prompt(self, user_input):
        return f"""Improve the following translation:
        Source: {user_input['source']},
        Translation: {user_input['translation']}
        Evaluation: {user_input['evaluation']}
        """

In [6]:
class FinalEditor(Agent):
    def system_prompt(self):
        return """"You are an expert editor. You provide final drafts of translations based on the work of translators. 
        You take into account their input and provide the final translation, with no additional outputs."""
    
    def user_prompt(self, user_input):
        return f"""Provide a final draft output based on this draft and information from the translation team: {user_input}
        """

In [7]:
prepro = Preprocessor()
translator = Translator()
eval = Evaluator()
post = PostEditor()
final = FinalEditor()

In [8]:
tib = """ །​རྒྱ་གར་སྐད་དུ། སརྦ་ཏ་ཐཱ་ག་ཏ་ཀཱ་ཡ་ཝཱཀྩིཏྟི་ཀྲྀཥྞ་ཡ་མཱ་རི་ནཱ་མ་ཏནྟྲ། བོད་སྐད་དུ། དེ་བཞིན་གཤེགས་པ་ཐམས་ཅད་ཀྱི་སྐུ་གསུང་ཐུགས་གཤིན་རྗེ་གཤེད་ནག་པོ་ཞེས་བྱ་བའི་རྒྱུད། འཕགས་པ་འཇམ་དཔལ་གཤིན་རྗེའི་གཤེད་ལ་ཕྱག་འཚལ་ལོ། །​འདི་སྐད་བདག་གིས་ཐོས་པ་དུས་གཅིག་ན། བཅོམ་ལྡན་འདས་དེ་བཞིན་གཤེགས་པ་ཐམས་ཅད་ཀྱི་སྐུ་དང་གསུང་དང་ཐུགས་རྡོ་རྗེའི་བཙུན་མོ་ཐམས་ཅད་ཀྱི་བྷ་ག་རྣམས་ལ་བཞུགས་ཏེ། གཏི་མུག་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་དང་། ཕྲ་མོ་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་དང་། འདོད་ཆགས་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་དང་། ཕྲག་དོག་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་དང་། ཞེ་སྡང་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་དང་། ཐོ་བ་གཤིན་རྗེའི་གཤེད་དང་། དབྱུག་པ་གཤིན་རྗེའི་གཤེད་དང་། པདྨ་གཤིན་རྗེའི་གཤེད་དང་། རལ་གྲི་གཤིན་རྗེའི་གཤེད་དང་། མཚམས་རྣམས་སུ་རྡོ་རྗེ་ཙ་རྩི་ཀཱ་དང་། རྡོ་རྗེ་ཕག་མོ་དང་། རྡོ་རྗེ་དབྱངས་ཅན་མ་དང་། རྡོ་རྗེ་གཽ་རཱི་མ་རྣམས་མཚམས་སུའོ། །​དེ་དག་ལ་སོགས་ཏེ་གཤིན་རྗེའི་གཤེད་ཀྱི་ཚོགས་ཆེན་པོ་དང་། ཐབས་ཅིག་ཏུ་བཞུགས་སོ། །​དེ་ནས་ཕྱག་ན་རྡོ་རྗེས་བཅོམ་ལྡན་འདས་རྡོ་རྗེ་དེ་བཞིན་གཤེགས་པ་ཐམས་ཅད་ཀྱི་བདག་པོ་ལ་གསོལ་བ་བཏབ་པ་དང་། དེ་ནས་བཅོམ་ལྡན་འདས་དེ་བཞིན་གཤེགས་པ་ཐམས་ཅད་ཀྱི་བདག་པོས་བདུད་ཐམས་ཅད་རྣམ་པར་འཇོམས་པའི་རྡོ་རྗེ་ཞེས་བྱ་བའི་ཏིང་ངེ་འཛིན་འདི་ཉིད་ཀྱི་སྐུ་གསུང་ཐུགས་རྡོ་རྗེ་ལས་ངེས་པར་བཏོན་ཏོ། །​ཟླ་བ་རྡོ་རྗེ་སྦྱོར་བ་ཡིས། །​གཤིན་རྗེ་མཐར་བྱེད་བསྒོམ་པར་བྱ། །​བདུད་རྣམས་ཞི་བར་བྱ་ཕྱིར་དང་། །​ཞེ་སྡང་ཐམས་ཅད་བསལ་བའི་ཕྱིར། །​འོད་ཟེར་ལྔ་ཡིས་འཁྲིགས་པ་ཡིས། །​རྡོ་རྗེ་བསྲུང་བའི་དོན་དུ་བསྒོམ། །​ཕྱི་མཚམས་རྡོ་རྗེ་ས་གཞི་དང་། །​ར་བ་དྲ་བ་དེ་བཞིན་ནོ། །​དེ་ནས་བཅོམ་ལྡན་འདས་དེ་བཞིན་གཤེགས་པ་སྐྱེད་པ་པོ་བདུད་འདུལ་བ་ཞེས་བྱ་བའི་ཏིང་ངེ་འཛིན་ལ་སྙོམས་པར་ཞུགས་ནས་གཏི་མུག་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་ལ་སོགས་པ་ཐམས་ཅད་ཀྱི་ས་བོན་འདི་ཉིད་ཀྱི་སྐུ་གསུང་ཐུགས་རྡོ་རྗེ་རྣམས་ལ་ངེས་པར་ཕྱུང་བ། དབུས་སུ་ཡ་དང་ཀྵེ་མ་མེད་ཡཙྪ་ནི་རཱ་ཛཱ་ས་དོ་རུ་ཎ་ཡོ་ནི་རའོ། །​ཡ་སོགས་ཡི་གེ་དང་པོ་ལས། །​གཤིན་རྗེའི་གཤེད་དུ་གྱུར་པ་ཡིན། །​ཡི་གེ་ཀྵེ་ནི་གཏི་མུག་བརྗོད།"""

In [9]:
internal_work = []

In [10]:
internal_work.append(tib)

prepro_notes = prepro.chat(user_input=tib)
internal_work.append(prepro_notes)

In [11]:
response = translator.chat({'source':tib, 'prepro_notes':prepro_notes})
internal_work.append(response)

In [12]:
evaluation = eval.chat({'source': tib, 'target': response})
internal_work.append(evaluation)

In [13]:
post = post.chat({'source': tib, 'translation': response , 'evaluation':evaluation})
internal_work.append(post)

In [14]:
final = final.chat(post)
internal_work.append(final)

In [15]:
print(final)

The Sanskrit name is Sarva Tathagata Kaya Vakti Chitti Krishna Yamari Namañtra. In Tibetan, this is known as the lineage of Vajradhatvishvara, the Black Wrathful One, the One Who Transcends. I pay homage to Jambhala, the Wrathful One.

Upon hearing this, the Compassionate One, the One Who Transcends, resides within the body, speech, and mind of Vajradhatvishvara, and in the consort of all Vajrasattvas. This lineage encompasses Vajradhatvishvara himself, as well as Phakmo Vajradhatvishvara, Madhyamika Vajradhatvishvara, Marici Vajradhatvishvara, Htimukha Vajradhatvishvara, Thobha Vajradhatvishvara, Drukpa Vajradhatvishvara, Padma Vajradhatvishvara, Raldri Vajradhatvishvara, and the consorts of the Vajrasattvas in the middle.

These and many others constitute the great assemblies of Vajradhatvishvara's lineage, and he practices in a single posture. From this, holding a vajra in hand, Vajradhatvishvara makes an offering to the Lord, the Sustainer of the Compassionate One, the One Who Tran

In [16]:
for _ in internal_work:
    print(_)

 །​རྒྱ་གར་སྐད་དུ། སརྦ་ཏ་ཐཱ་ག་ཏ་ཀཱ་ཡ་ཝཱཀྩིཏྟི་ཀྲྀཥྞ་ཡ་མཱ་རི་ནཱ་མ་ཏནྟྲ། བོད་སྐད་དུ། དེ་བཞིན་གཤེགས་པ་ཐམས་ཅད་ཀྱི་སྐུ་གསུང་ཐུགས་གཤིན་རྗེ་གཤེད་ནག་པོ་ཞེས་བྱ་བའི་རྒྱུད། འཕགས་པ་འཇམ་དཔལ་གཤིན་རྗེའི་གཤེད་ལ་ཕྱག་འཚལ་ལོ། །​འདི་སྐད་བདག་གིས་ཐོས་པ་དུས་གཅིག་ན། བཅོམ་ལྡན་འདས་དེ་བཞིན་གཤེགས་པ་ཐམས་ཅད་ཀྱི་སྐུ་དང་གསུང་དང་ཐུགས་རྡོ་རྗེའི་བཙུན་མོ་ཐམས་ཅད་ཀྱི་བྷ་ག་རྣམས་ལ་བཞུགས་ཏེ། གཏི་མུག་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་དང་། ཕྲ་མོ་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་དང་། འདོད་ཆགས་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་དང་། ཕྲག་དོག་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་དང་། ཞེ་སྡང་རྡོ་རྗེ་གཤིན་རྗེའི་གཤེད་དང་། ཐོ་བ་གཤིན་རྗེའི་གཤེད་དང་། དབྱུག་པ་གཤིན་རྗེའི་གཤེད་དང་། པདྨ་གཤིན་རྗེའི་གཤེད་དང་། རལ་གྲི་གཤིན་རྗེའི་གཤེད་དང་། མཚམས་རྣམས་སུ་རྡོ་རྗེ་ཙ་རྩི་ཀཱ་དང་། རྡོ་རྗེ་ཕག་མོ་དང་། རྡོ་རྗེ་དབྱངས་ཅན་མ་དང་། རྡོ་རྗེ་གཽ་རཱི་མ་རྣམས་མཚམས་སུའོ། །​དེ་དག་ལ་སོགས་ཏེ་གཤིན་རྗེའི་གཤེད་ཀྱི་ཚོགས་ཆེན་པོ་དང་། ཐབས་ཅིག་ཏུ་བཞུགས་སོ། །​དེ་ནས་ཕྱག་ན་རྡོ་རྗེས་བཅོམ་ལྡན་འདས་རྡོ་རྗེ་དེ་བཞིན་གཤེགས་པ་ཐམས་ཅད་ཀྱི་བདག་པོ་ལ་གསོལ་བ་བཏབ་པ་དང་། དེ་ནས་བཅོམ་ལྡན་འདས་དེ་བཞིན་གཤེགས་པ་ཐམས་ཅད་ཀྱི་བདག་པོས་བདུད་ཐམས་ཅད་རྣམ་པ